In [1]:
import threading
from etl.etl_run import ETLRun

from models import ParentPair, ChildPair

from sqlalchemy import create_engine
from db_constants import Base
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)

In [2]:
stop_event = threading.Event()
data_loader = DataLoader()
etl = ETLRun(stop_event, pairs_limit=150)
etl.start_threads()

try:
    # Keep the main thread running
    while True:
        pass
except KeyboardInterrupt:
    # On KeyboardInterrupt, stop the WebSocketThread
    stop_event.set()
    etl.stop_threads()


Connected BTCUSDT
Connected ETHBTC
Connected ETHTUSD
Connected ETHBTC
Connected ETHBTC
Connected ETHUSDT
Connected ETHPAX
Connected BTCTUSD
Connected ETHBUSD
Connected BTCUSDC
Connected BTCPAX
Connected ETHBTC
Connected ETHBTC
Connected BTCRUB
Connected ETHUSDC
Connected ETHBTC
Connected ETHRUB
Connected ETHBTC
Connected ETHTRY
Connected ETHBTC
Connected ETHBTC
Connected BTCTRY
Connected ETHBTC
Connected BTCBUSD
Connected BTCEUR
Connected ETHEUR
Connected ETHZAR
Connected ETHBKRW
Connected ETHBTC
Connected BTCBKRW
Connected ETHBTC
Connected ETHBTC
Connected BTCBIDR
Connected ETHBIDR
Connected ETHAUD
Connected BTCZAR
Connected ETHNGN
Connected BTCGBP
Connected ETHGBP
Connected BTCAUD
Connected ETHBTC
Connected BTCDAI
Connected ETHDAI
Connected BTCNGN
Connected ETHBRL
Connected BTCBRL
Connected ETHBTC
Connected ETHBTC
Connected ETHUST
Connected ETHBTC
Connected BTCUSDP
Connected ETHUAH
Connected ETHUSDP
Connected BTCUAH
Connected ETHBTC
Connected ETHBTC
Connected ETHFDUSD
Connected BTCPL